In [ ]:
# necessary imports
import torch
import pickle
from openood.evaluation_api import Evaluator
from openood.networks import ResNet18_32x32

In [ ]:
# 파일 경로
file_path = '.pth'

# 모델 정의
net = ResNet18_32x32(num_classes=10)

# 저장된 모델 로드
checkpoint = torch.load(file_path, map_location=torch.device("cuda"))  # CPU에서도 로드 가능하도록 설정


# 모델 가중치 로드
net.load_state_dict(checkpoint['state_dict'])

# 모델을 GPU로 이동
net.cuda()
net.eval()

In [ ]:
OOD_list = ['msp', 'odin','ebo','gradnorm','react','mls','klm','vim','knn','dice','rankfeat','ash','she','mds','rmds','gram','mds_ensemble','temp_scaling','openmax']

res = []
for method in OOD_list:
    print(f"-------------------{method}----------------------")
    postprocessor_name = method
    try:
        evaluator = Evaluator(
            net,
            id_name='cifar10',                     
            data_root='./data',                    
            config_root=None,                      
            preprocessor=None,                     
            postprocessor_name=postprocessor_name, 
            postprocessor=None,                    
            batch_size=200,                        
            shuffle=False,
            num_workers=2)                         

        metrics = evaluator.eval_ood(fsood=False)
        near, far = metrics['AUROC']['nearood'], metrics['AUROC']['farood']
        accuracy = metrics['ACC']['nearood']

    except Exception as e:
        print(f"Error occurred with method {method}: {e}")
        near, far = 0, 0
        accuracy = 0

    res_each = [method, near, far]
    res.append(res_each)

res.append(accuracy)


# 지정된 파일 경로
file_path = '.pkl'

# res 리스트를 pickle 파일로 저장
with open(file_path, 'wb') as f:
    pickle.dump(res, f)

print(f"Results have been saved to {file_path}")